In [12]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [13]:
data = pd.read_csv('./digit-recognizer/train.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [15]:
X_train[:, 0].shape

(784,)

<iframe src="https://www.kaggle.com/embed/wwsalmon/simple-mnist-nn-from-scratch-numpy-no-tf-keras?cellIds=5&kernelSessionId=47652503" height="300" style="margin: 0 auto; width: 100%; max-width: 950px;" frameborder="0" scrolling="auto" title="Simple MNIST NN from scratch (numpy, no TF/Keras)"></iframe>

In [16]:
def init_params(size):
    W1 = np.random.rand(10, size) - 0.5
    b1 = np.random.rand(10,1) - 0.5
    W2 = np.random.rand(10,10) - 0.5
    b2 = np.random.rand(10,1) - 0.5
    
    return W1,b1,W2,b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    exp = np.exp(Z - np.max(Z))
    return exp / exp.sum(axis=0)

def forward_propagation(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2
    
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    
    return one_hot_Y
    
def deriv_ReLU(Z):
    return Z > 0
    
def back_propagation(Z1, A1, Z2, A2, W1, W2, X, Y, m):
    one_hot_Y = one_hot(Y)
    dZ2 = 2 * (A2 - one_hot_Y)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, 1)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, 1)
    
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * np.reshape(db1, (10,1))
    W2 -= alpha * dW2
    b2 -= alpha * np.reshape(db2, (10,1))
    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [17]:
def gradient_descent(X, Y, iterations, alpha):
    size, m = X.shape
    W1, b1, W2, b2 = init_params(size)
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_propagation(Z1, A1, Z2, A2, W1, W2, X, Y, m)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if(i % 50) == 0:
            print("Iteration:", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [18]:
X_train.shape, Y_train.shape

((784, 41000), (41000,))

In [19]:
one_hot(Y_train)

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.1)

Iteration: 0
[3 8 8 ... 3 3 0] [0 8 4 ... 7 0 3]
Accuracy:  0.0905609756097561
Iteration: 50
[0 3 4 ... 7 0 0] [0 8 4 ... 7 0 3]
Accuracy:  0.6244878048780488
Iteration: 100
[0 8 4 ... 7 0 5] [0 8 4 ... 7 0 3]
Accuracy:  0.7465853658536585
Iteration: 150
[0 3 4 ... 7 0 5] [0 8 4 ... 7 0 3]
Accuracy:  0.7735609756097561
Iteration: 200
[0 8 4 ... 7 0 5] [0 8 4 ... 7 0 3]
Accuracy:  0.8137560975609756
Iteration: 250
[0 8 4 ... 7 0 5] [0 8 4 ... 7 0 3]
Accuracy:  0.835780487804878
Iteration: 300
[0 8 4 ... 7 0 5] [0 8 4 ... 7 0 3]
Accuracy:  0.8487560975609756
Iteration: 350
[0 8 4 ... 7 0 3] [0 8 4 ... 7 0 3]
Accuracy:  0.8582682926829268
Iteration: 400
[0 8 4 ... 7 0 3] [0 8 4 ... 7 0 3]
Accuracy:  0.865780487804878
Iteration: 450
[0 8 4 ... 7 0 3] [0 8 4 ... 7 0 3]
Accuracy:  0.8715365853658537
